#Fine-tune AraBERT with Fast-BERT library
https://github.com/kaushaltrivedi/fast-ber

##Get Requirements

In [1]:
!git clone https://github.com/aub-mind/arabert
!pip install PyArabic farasapy fast-bert

Cloning into 'arabert'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 279 (delta 34), reused 38 (delta 15), pack-reused 214
Receiving objects: 100% (279/279), 3.68 MiB | 8.77 MiB/s, done.
Resolving deltas: 100% (157/157), done.
     |████████████████████████████████| 91 kB 482 kB/s 
     |████████████████████████████████| 43 kB 703 kB/s 
     |████████████████████████████████| 3.0 MB 1.3 MB/s 
     |████████████████████████████████| 769 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=d32b1b691b19a0dce8a199d5029a92bbc3a622b7d88763e4227a5f7d96034f8c
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.3
    Uninstalling tokenizers-0.9.3:
      Successfully uninstalled token

##Prepare the data

In [2]:
import pandas as pd
#from py4j.java_gateway import JavaGateway
from farasa.segmenter import FarasaSegmenter
from arabert.preprocess_arabert import preprocess
from sklearn.model_selection import train_test_split

# !pkill "java"
# gateway = JavaGateway.launch_gateway(classpath='./FarasaSegmenterJar.jar')
# farasa_segmenter = gateway.jvm.com.qcri.farasa.segmenter.Farasa()

farasa_segmenter = FarasaSegmenter(interactive=True)

df_AJGT = pd.read_excel('../input/tunisian-texts/tun.xlsx',header=0)
df_AJGT['data_labels'] = df_AJGT['data_labels'].astype(str)

DATA_COLUMN = 'texts'
LABEL_COLUMN = 'data_labels'

df_AJGT = df_AJGT[['texts', 'data_labels']]
df_AJGT.columns = [DATA_COLUMN, LABEL_COLUMN]

label_map = {
    'Negative' : 0,
    'Positive' : 1
}

df_AJGT[DATA_COLUMN] = df_AJGT[DATA_COLUMN].apply(lambda x: preprocess(x, do_farasa_tokenization=True , farasa=farasa_segmenter, use_farasapy = True))
# df_AJGT[LABEL_COLUMN] = df_AJGT[LABEL_COLUMN].apply(lambda x: label_map[x])

train_AJGT, test_AJGT = train_test_split(df_AJGT, test_size=0.2,random_state=42)
!mkdir data
train_AJGT.to_csv("./train.csv",index=True,columns=train_AJGT.columns,sep=',',header=True)
test_AJGT.to_csv("./dev.csv",index=True,columns=test_AJGT.columns,sep=',',header=True)
with open('./labels.csv','w') as f:
    f.write("\n".join(df_AJGT['data_labels'].unique()))

100%|██████████| 241M/241M [00:15<00:00, 12.9MiB/s]

[2020-12-10 17:05:11,183 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


100%|██████████| 241M/241M [00:30<00:00, 12.9MiB/s]

In [3]:
with open('./labels.csv','w') as f:
    f.write("\n".join(df_AJGT['data_labels'].unique()))

##Create a DataBunch Object:
see https://github.com/kaushaltrivedi/fast-bert#text-classification

In [4]:
from fast_bert.data_cls import BertDataBunch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')

databunch = BertDataBunch('./', './',
                          tokenizer=tokenizer,
                          train_file='train.csv',
                          val_file='dev.csv',
                          label_file='labels.csv',
                          text_col='texts',
                          label_col='data_labels',
                          batch_size_per_gpu=16,
                          max_seq_length=256,
                          multi_gpu=True,
                          multi_label=False,
                          model_type='bert',
                          )

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


##Create the Learner Object
see https://github.com/kaushaltrivedi/fast-bert#2-create-a-learner-object

In [5]:
import logging
import torch

from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device_cuda = torch.device("cuda")
metrics = [{'name': 'accuracy', 'function': accuracy}]

!mkdir 'output'
learner = BertLearner.from_pretrained_model(
						databunch,
						pretrained_path='aubmindlab/bert-base-arabert',
						metrics=metrics,
						device=device_cuda,
						logger=logger,
						output_dir='output',
						finetuned_wgts_path=None,
						warmup_steps=30,
						multi_gpu=False,
						is_fp16=False,
						multi_label=False,
						logging_steps=0)

##Start Training and Validating

In [6]:
learner.fit(epochs=3,
			lr=2e-5,
			validate=True, 	# Evaluate the model after each epoch
			schedule_type="warmup_linear",
			optimizer_type="adamw")

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


(4923, 0.19957116219492618)

##You can see the output using tensorboard

In [7]:
%load_ext tensorboard
%tensorboard --logdir './output/tensorboard'

In [8]:
learner.save_model()

In [9]:
#Predictions
#Call predict_batch method on the learner object that contains the trained model.

texts = [
  "مخيب عيشتكم وملا حالة هالكورونا",
  "البلاد مشات يا لطيف"
]

predictions = learner.predict_batch(texts)
from fast_bert.prediction import BertClassificationPredictor

predictor = BertClassificationPredictor('aubmindlab/bert-base-arabert','./')

# Single prediction
single_prediction = predictor.predict("مخيب عيشتكم وملا حالة هالكورونا")

# Batch predictions
texts = [
  "مخيب عيشتكم وملا حالة هالكورونا",
  "البلاد مشات يا لطيف"
]

multiple_predictions = predictor.predict(texts)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [10]:
l = pd.read_csv('./labels.csv')
l

,1
0,0
